In [1]:
import re
import datetime
import pyspark

from pyspark.sql import Row
from pyspark import SparkContext
import sys
import os
from operator import add

sc = pyspark.SparkContext('local[*]')

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)


In [2]:
df = sqlContext.createDataFrame([('Anthony', 10), ('Julia', 20), ('Fred', 5)], ('name', 'count'))
df.show() # This line does not work. Please comment it out later.
df.select(df['count']).show()

+-------+-----+
|   name|count|
+-------+-----+
|Anthony|   10|
|  Julia|   20|
|   Fred|    5|
+-------+-----+

+-----+
|count|
+-----+
|   10|
|   20|
|    5|
+-----+



In [3]:
base_df = sqlContext.read.text('data/usask_access_sanity_check_log')
# Let's look at the schema
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
print(base_df.count())
base_df.show(truncate=False)


16
+-----------------------------------------------------------------------------------------------+
|value                                                                                          |
+-----------------------------------------------------------------------------------------------+
|skul2.usask.ca - - [15/Jun/1995:13:59:11 -0600] "GET /images/logo.gif HTTP/1.0" 200 2273       |
|bell.usask.ca - - [15/Jun/1995:13:59:32 -0600] "GET /images/logo.gif HTTP/1.0" 304 0           |
|142.99.48.34 - - [15/Jun/1995:13:59:36 -0600] "GET /images/logo.gif HTTP/1.0" 200 2273         |
|villi.usask.ca - - [15/Jun/1995:14:00:49 -0600] "GET /images/logo.gif HTTP/1.0" 304 0          |
|chemeng03 - - [15/Jun/1995:14:02:07 -0600] "GET /images/logo.gif HTTP/1.0" 200 2273            |
|pukatea.its.vuw.ac.nz - - [15/Jun/1995:14:03:50 -0600] "GET /images/logo.gif HTTP/1.0" 200 2273|
|gp108008.usask.ca - - [15/Jun/1995:14:04:10 -0600] "GET /images/logo.gif HTTP/1.0" 304 0       |
|villi.usask.ca -

In [5]:
from pyspark.sql.functions import split, regexp_extract
parsed_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
parsed_df.show(truncate=False)

+----------------------+--------------------------+----------------+------+------------+
|host                  |timestamp                 |path            |status|content_size|
+----------------------+--------------------------+----------------+------+------------+
|skul2.usask.ca        |15/Jun/1995:13:59:11 -0600|/images/logo.gif|200   |2273        |
|bell.usask.ca         |15/Jun/1995:13:59:32 -0600|/images/logo.gif|304   |0           |
|142.99.48.34          |15/Jun/1995:13:59:36 -0600|/images/logo.gif|200   |2273        |
|villi.usask.ca        |15/Jun/1995:14:00:49 -0600|/images/logo.gif|304   |0           |
|chemeng03             |15/Jun/1995:14:02:07 -0600|/images/logo.gif|200   |2273        |
|pukatea.its.vuw.ac.nz |15/Jun/1995:14:03:50 -0600|/images/logo.gif|200   |2273        |
|gp108008.usask.ca     |15/Jun/1995:14:04:10 -0600|/images/logo.gif|304   |0           |
|villi.usask.ca        |15/Jun/1995:14:04:14 -0600|/images/logo.gif|304   |0           |
|gp108008.usask.ca   

In [6]:
df = parsed_df.filter(parsed_df['path'].isNull())
df.show()
df.count()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
+----+---------+----+------+------------+



0

In [7]:
df = parsed_df.filter(parsed_df['content_size'].isNull())
df.show()
df.count()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
+----+---------+----+------+------------+



0

Note: In the expression below, ~ means "not".

In [10]:
df = parsed_df.filter(~parsed_df['content_size'].isNull())
df.show()
df.count()

+--------------------+--------------------+----------------+------+------------+
|                host|           timestamp|            path|status|content_size|
+--------------------+--------------------+----------------+------+------------+
|     skul2.usask.ca |15/Jun/1995:13:59...|/images/logo.gif|   200|        2273|
|      bell.usask.ca |15/Jun/1995:13:59...|/images/logo.gif|   304|           0|
|       142.99.48.34 |15/Jun/1995:13:59...|/images/logo.gif|   200|        2273|
|     villi.usask.ca |15/Jun/1995:14:00...|/images/logo.gif|   304|           0|
|          chemeng03 |15/Jun/1995:14:02...|/images/logo.gif|   200|        2273|
|pukatea.its.vuw.a...|15/Jun/1995:14:03...|/images/logo.gif|   200|        2273|
|  gp108008.usask.ca |15/Jun/1995:14:04...|/images/logo.gif|   304|           0|
|     villi.usask.ca |15/Jun/1995:14:04...|/images/logo.gif|   304|           0|
|  gp108008.usask.ca |15/Jun/1995:14:05...|/images/logo.gif|   304|           0|
|  gp108008.usask.ca |15/Jun

16

In [12]:
filtered_df = parsed_df.filter(~ (parsed_df['host'].isNull() |
                                 parsed_df['timestamp'].isNull() |
                                 parsed_df['path'].isNull() |
                                 parsed_df['status'].isNull() |
                                 parsed_df['content_size'].isNull()))
filtered_df.count()

16

In [13]:
from pyspark.sql import functions as sqlFunctions
content_size_stats =  (filtered_df
                       .agg(sqlFunctions.min(filtered_df['content_size']),
                            sqlFunctions.avg(filtered_df['content_size']),
                            sqlFunctions.max(filtered_df['content_size']))
                       .first())

print ('Using SQL functions:')
print ('Content Size Avg: {1:,.2f}; Min: {0:.2f}; Max: {2:,.0f}'.format(*content_size_stats))

Using SQL functions:
Content Size Avg: 994.44; Min: 0.00; Max: 2,273


In [14]:
status_to_count_df =(filtered_df
                     .groupBy('status')
                     .count()
                     .sort('status')
                     .cache())

status_to_count_length = status_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

Found 4 response codes
+------+-----+
|status|count|
+------+-----+
|   200|    7|
|   304|    6|
|   500|    1|
|   502|    2|
+------+-----+



In [15]:

status_to_count_df =(filtered_df
                     .groupBy('path', 'status')
                     .count()
                     .orderBy('path' )
                     .cache())

status_to_count_length = status_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

Found 4 response codes
+----------------+------+-----+
|            path|status|count|
+----------------+------+-----+
|/images/logo.gif|   502|    2|
|/images/logo.gif|   304|    6|
|/images/logo.gif|   500|    1|
|/images/logo.gif|   200|    7|
+----------------+------+-----+



In [16]:

status_to_count_df =(filtered_df
                     .groupBy('path')
                     .count()
                     .sort('path', ascending=False)
                     .cache())

status_to_count_length = status_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

Found 1 response codes
+----------------+-----+
|            path|count|
+----------------+-----+
|/images/logo.gif|   16|
+----------------+-----+



In [ ]:
status_to_count_df =(filtered_df
                     .groupBy('path', 'status')
                     .count()
                     .sort('count', ascending=False)
                     .cache())

status_to_count_length = status_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
status_to_count_df.show()

In [ ]:
not200_df = filtered_df.filter(filtered_df['status'] != 200)
not200_df.show()

In [ ]:
not200_df_to_count_df =(not200_df
                     .groupBy('path', 'status')
                     .count()
                     .sort('status', ascending=False)
                     .cache())

status_to_count_length = not200_df_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
not200_df_to_count_df.show()

In [ ]:
not200_df_to_count_df =(not200_df
                     .groupBy('path')
                     .count()
                     .sort('count', ascending=False)
                     .cache())

status_to_count_length = not200_df_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
not200_df_to_count_df.show()

In [ ]:

    
status_to_count_df =(filtered_df
                     .groupBy('host')
                     .agg(sqlFunctions.min(filtered_df['content_size']),
                            sqlFunctions.avg(filtered_df['content_size']),
                            sqlFunctions.max(filtered_df['content_size']),
                            sqlFunctions.count(filtered_df['host']))
                     .sort('count(host)', ascending=False)
                     .cache())

status_to_count_length = status_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
status_to_count_df.show()    

In [ ]:
   status_to_count_df =(not200_df
                     .groupBy('host', 'status')
                     .agg(sqlFunctions.min(filtered_df['content_size']),
                            sqlFunctions.avg(filtered_df['content_size']),
                            sqlFunctions.max(filtered_df['content_size']),
                            sqlFunctions.count(filtered_df['host']))
                     .sort('host', ascending=False)
                     .cache())

status_to_count_length = status_to_count_df.count()
print ('Found %d response codes' % status_to_count_length)
status_to_count_df.show() 